### simple Gen Ai app Using Langchain

In [5]:
import os 
from dotenv import load_dotenv
load_dotenv()


os.environ["OPENAI_API_KEY"]= os.getenv("OPEN_API_KEY")
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"
## langsmith Tracking 
os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]= "true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["OPENAI_MODEL_NAME"]="openai/gpt-oss-20b:free"
 

In [6]:
## Data ingestion -- from the website we need to scrape the data 

from langchain_community.document_loaders import WebBaseLoader



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
loader = WebBaseLoader("https://docs.langchain.com/langsmith/administration-overview")

In [8]:
loader 

In [9]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/administration-overview', 'title': 'Overview - Docs by LangChain', 'language': 'en'}, page_content='Overview - Docs by LangChainSkip to main content🚀 Share how you\'re building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationOverviewGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementReferenceLangSmith Python SDKLangSmith JS/TS SDKLangGraph Python SDKLangGraph JS/TS SDKLangSmith APIAPI reference for LangSmith DeploymentAdditional resourcesReleases & changelogsData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQOn this pageResource HierarchyOrganization

In [10]:
## Load Data ---> Docs ---> Divide out text into chunks --->text --->vectors ----> Vector Embeddings --->>> vector store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
splitter = RecursiveCharacterTextSplitter(chunk_size=1000 , chunk_overlap=200)
text_split = splitter.split_documents(docs)

In [11]:
text_split

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/administration-overview', 'title': 'Overview - Docs by LangChain', 'language': 'en'}, page_content="Overview - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationOverviewGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementReferenceLangSmith Python SDKLangSmith JS/TS SDKLangGraph Python SDKLangGraph JS/TS SDKLangSmith APIAPI reference for LangSmith DeploymentAdditional resourcesReleases & changelogsData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQOn this pageResource HierarchyOrganizations

In [12]:
### Embedding techneqes

from langchain_community.embeddings import OpenAIEmbeddings
embedding = OpenAIEmbeddings(
    model = "openai/text-embedding-3-large"
)

C:\Users\hp\AppData\Local\Temp\ipykernel_25132\3185047310.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(


In [13]:
from langchain_community.vectorstores import FAISS
DB_vector = FAISS.from_documents(text_split,embedding)

In [14]:
DB_vector

In [15]:
## Query from vector store DB
query = "Users can be members of one or more organizations "
result = DB_vector.similarity_search(query)

result[0].page_content

'\u200bUser Management and RBAC\n\u200bUsers\nA user is a person who has access to LangSmith. Users can be members of one or more organizations and workspaces within those organizations.\nOrganization members are managed in organization settings:\n\nAnd workspace members are managed in workspace settings:'

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x0000025452964C10> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000025452965B10> root_client=<openai.OpenAI object at 0x0000025452957850> root_async_client=<openai.AsyncOpenAI object at 0x0000025452965690> model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://openrouter.ai/api/v1'


In [18]:

# correct import (the combine_documents chain lives in langchain.chains)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """ 
Ansewer the following question based only on the provided context : 
<context>
{context}

</context>
"""
)

documet_chain = create_stuff_documents_chain(llm,prompt)
documet_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' \nAnsewer the following question based only on the provided context : \n<context>\n{context}\n\n</context>\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000025452964C10>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000025452965B10>, root_client=<openai.OpenAI object at 0x0000025452957850>, root_async_client=<openai.AsyncOpenAI object at 0x0000025452965690>, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://openrouter.ai/api/v1')
| StrOut

In [19]:
from langchain_core.documents import Document
documet_chain.invoke({
    "input": "Users can be members of one or more organizations",
    "context":[Document(page_content="Users can be members of one or more organizations workspaces within those organizations. Organization members are managed in organization settings:")]
})

'Q: How are users managed within organizations?\nA: Organization members are managed in organization settings.'

* However,we want the documents to first come from the retriever we just set up. That way, we can use the retriever do dynamically select the most relevant documents and pass those in for a given question 

In [20]:
### input ---> Retriever --->vectorstore Db  

DB_vector

In [25]:
retrever = DB_vector.as_retriever()
from langchain.chains import create_retrieval_chain
retrievel_chain =create_retrieval_chain(retrever , documet_chain)

In [28]:
## GEt the response from the LLM

response = retrievel_chain.invoke({"input":"Personal Access Tokens (PATs) are used to authenticate requests"})
response['answer']

'Service keys are used to authenticate requests to the LangSmith API on behalf of a service account and can be scoped to a single workspace, multiple workspaces, or the entire organization. They are prefixed with lsv2_sk_. Only admins can create service keys.'

In [29]:
response

{'input': 'Personal Access Tokens (PATs) are used to authenticate requests',
 'context': [Document(id='fad4d23b-9285-4a16-94d1-0126850c0d2c', metadata={'source': 'https://docs.langchain.com/langsmith/administration-overview', 'title': 'Overview - Docs by LangChain', 'language': 'en'}, page_content='PATs are prefixed with lsv2_pt_\n\u200bService keys\nService keys are similar to PATs, but are used to authenticate requests to the LangSmith API on behalf of a service account. Only admins can create service keys. We recommend using these for applications / services that need to interact with the LangSmith API, such as LangGraph agents or other integrations. Service keys may be scoped to a single workspace, multiple workspaces, or the entire organization, and can be used to authenticate requests to the LangSmith API for whichever workspace(s) it has access to.\nService keys are prefixed with lsv2_sk_\nUse the X-Tenant-Id header to specify the target workspace.\nWhen using PATs: If this head

In [30]:
response['context']

[Document(id='fad4d23b-9285-4a16-94d1-0126850c0d2c', metadata={'source': 'https://docs.langchain.com/langsmith/administration-overview', 'title': 'Overview - Docs by LangChain', 'language': 'en'}, page_content='PATs are prefixed with lsv2_pt_\n\u200bService keys\nService keys are similar to PATs, but are used to authenticate requests to the LangSmith API on behalf of a service account. Only admins can create service keys. We recommend using these for applications / services that need to interact with the LangSmith API, such as LangGraph agents or other integrations. Service keys may be scoped to a single workspace, multiple workspaces, or the entire organization, and can be used to authenticate requests to the LangSmith API for whichever workspace(s) it has access to.\nService keys are prefixed with lsv2_sk_\nUse the X-Tenant-Id header to specify the target workspace.\nWhen using PATs: If this header is omitted, requests will run against the default workspace associated with the key.\n